In [11]:
# Python implementation of:
# void testCircularAzimuthElevation() 
# from: KalmanEstimatorTest.java

In [39]:
%matplotlib inline

from math import radians, pi
import pandas as pd
import numpy as np
import plotly.express as px


import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()


from org.orekit.orbits import KeplerianOrbit, PositionAngle
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate, TimeScalesFactory, TimeScale
from org.orekit.utils import Constants
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import LofOffset
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.forces.drag import IsotropicDrag
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.frames import FramesFactory, LOFType, EOPHistory
from org.orekit.models.earth.atmosphere import HarrisPriester
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.propagation import PropagationType, SpacecraftState
from org.orekit.propagation.events import DateDetector, EventEnablingPredicateFilter, PythonEnablingPredicate,\
     PositionAngleDetector, PythonEventDetector, AbstractDetector, EventDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder
from org.orekit.propagation.semianalytical.dsst.forces import DSSTAtmosphericDrag, DSSTNewtonianAttraction, DSSTZonal
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from orekit.pyhelpers import absolutedate_to_datetime
from org.hipparchus.linear import RealMatrix
from org.hipparchus.linear import QRDecomposer, AbstractRealMatrix
from org.hipparchus.optim.nonlinear.vector.leastsquares import GaussNewtonOptimizer
from org.orekit.estimation.leastsquares import BatchLSEstimator
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.propagation.conversion import AbstractPropagatorBuilder

from org.orekit.estimation.measurements import AngularAzEl
from org.orekit.estimation.measurements import ObservableSatellite
from org.orekit.estimation.measurements import GroundStation
from org.orekit.estimation.sequential import KalmanEstimatorBuilder
from org.orekit.estimation.sequential import KalmanEstimator
from org.orekit.estimation.sequential import KalmanEstimation
from org.orekit.estimation.measurements import *

from orekit.pyhelpers import absolutedate_to_datetime
from collections import namedtuple
from org.orekit.time import Month
from orekit import JArray
from orekit.pyhelpers import JArray_double2D

from org.orekit.estimation.sequential import KalmanEstimator, KalmanEstimatorBuilder, CovarianceMatrixProvider, ConstantProcessNoise
from org.orekit.orbits import Orbit


ImportError: cannot import name 'JArray_double2D' from 'orekit' (/home/whiskiss/anaconda3/lib/python3.8/site-packages/orekit/__init__.py)

In [29]:
utc = TimeScalesFactory.getUTC()

# Given by ADS
a = 7000.0 * 1000  
e = 0.0016 
i = radians(98.0)     
omega = radians(0.2)   # perigee argument
raan = radians(90.0)  # right ascension of ascending node
lv = radians(0.1)    # True anomaly

mass = 625.0

epochDate = AbsoluteDate(2020, 1, 1, 0, 0, 00.000, utc)
initialDate = epochDate

## Inertial frame where the satellite is defined
inertialFrame = FramesFactory.getEME2000()
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         inertialFrame)

## Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lv,
                              PositionAngle.TRUE,
                              inertialFrame, epochDate, Constants.WGS84_EARTH_MU)

In [30]:
propagator = KeplerianPropagator(initialOrbit)
propagator.getInitialState()

<SpacecraftState: SpacecraftState{orbit=Keplerian parameters: {a: 7000000.0; e: 0.0016; i: 98.0; pa: 0.2; raan: 90.0; v: 0.1;}, attitude=org.orekit.attitudes.Attitude@5dd1c9f2, mass=1000.0, additional={}}>

In [31]:
# Estimator creation

matrixDecomposer = QRDecomposer(1e-11)
optimizer = GaussNewtonOptimizer(matrixDecomposer, False)

builder = DSSTPropagatorBuilder(initialOrbit,
                                DormandPrince853IntegratorBuilder(1.0, 3600.0, 1000.0),
                                1.0, PropagationType.OSCULATING, PropagationType.OSCULATING)
builder.setMass(625.0)
propagator = builder.buildPropagator(builder.getSelectedNormalizedParameters())

#Check if builder or propagator needs to be passer and what is the difference
#estimator = KalmanEstimatorBuilder.build
#estimator.propagator.



In [33]:
def JArray_double2D(x, y):
    """Returns an JCC wrapped 2D double array

    Args:
        x: Number of rows in the array
        y: Number of columns in the array

    Note that the rows and columns are returned as objects and
    are likely needed to be casted manually.
    """

    arr = JArray('object')(x)

    for i in range(x):
        arr[i] = JArray('double')(y)

    return arr


In [34]:
initialOrbit = orbitType.convertType(initialOrbit)
dYdC =  JArray_double2D(6,6)
initialOrbit.getJacobianWrtCartesian(PositionAngle.TRUE, dYdC)
Jac = MatrixUtils.createRealMatrix(dYdC)

NameError: name 'MatrixUtils' is not defined

In [ ]:
// Jacobian of the orbital parameters w/r to Cartesian
        final Orbit initialOrbit = orbitType.convertType(context.initialOrbit);
        final double[][] dYdC = new double[6][6];
        initialOrbit.getJacobianWrtCartesian(PositionAngle.TRUE, dYdC);
        final RealMatrix Jac = MatrixUtils.createRealMatrix(dYdC);
        
        // Initial covariance matrix
        final RealMatrix initialP = Jac.multiply(cartesianP.multiply(Jac.transpose()));

        // Process noise matrix
        final RealMatrix cartesianQ = MatrixUtils.createRealDiagonalMatrix(new double [] {
            1.e-6, 1.e-6, 1.e-6, 1.e-12, 1.e-12, 1.e-12
        });
        final RealMatrix Q = Jac.multiply(cartesianQ.multiply(Jac.transpose()));
        
        // Build the Kalman filter
        final KalmanEstimator kalman = new KalmanEstimatorBuilder().
                        addPropagationConfiguration(propagatorBuilder, new ConstantProcessNoise(initialP, Q)).
                        build();
        
        // Filter the measurements and check the results
        final double   expectedDeltaPos  = 0.;
        final double   posEps            = 4.78e-7;
        final double   expectedDeltaVel  = 0.;
        final double   velEps            = 1.54e-10;
        final double[] expectedSigmasPos = {0.356902, 1.297507, 1.798551};
        final double   sigmaPosEps       = 1e-6;
        final double[] expectedSigmasVel = {2.468745e-4, 5.810027e-4, 3.887394e-4};
        final double   sigmaVelEps       = 1e-10;
        EstimationTestUtils.checkKalmanFit(context, kalman, measurements,
                                           refOrbit, positionAngle,
                                           expectedDeltaPos, posEps,
                                           expectedDeltaVel, velEps,
                                           expectedSigmasPos, sigmaPosEps,
                                           expectedSigmasVel, sigmaVelEps);
    }